In [1]:
cd /Users/jz-rolling/Desktop/OMEGA_GUI/notebooks/QY_Rv1830/Github_deposit/

/Users/jz-rolling/Desktop/OMEGA_GUI/notebooks/QY_Rv1830/Github_deposit


In [2]:
import momia2 as mo
from skimage import transform, filters, feature, measure, registration, io, color, morphology, restoration
import numpy as np
import pandas as pd
import read_roi as rr
import matplotlib.pyplot as plt
import glob,os
import pickle as pk
import tifffile
import cv2
from cv2_rolling_ball import subtract_background_rolling_ball
from colony_tracker import *

In [3]:
cd /Users/jz-rolling/Desktop/OMEGA_GUI/notebooks/QY_Rv1830/Github_deposit/1_Colony_tracing/

/Users/jz-rolling/Desktop/OMEGA_GUI/notebooks/QY_Rv1830/Github_deposit/1_Colony_tracing


In [4]:
# load reference points 
ref_x = np.array([2,3,4,5,6,7,8])
ref_y = np.array([2,3,4,5,6,7,8])
ref_xy = np.meshgrid(ref_x,ref_y)
ref_points = np.load('ref_grid_coords.npy')[ref_xy[1],ref_xy[0],:].reshape(49,2)

In [4]:
# define output path
dst = '/Users/jz-rolling/Desktop/Size_calculation/20220118/Projection_redo_20220119/'

In [9]:
# process plate 0
for f in sorted(glob.glob('/Users/jz-rolling/Desktop/Size_calculation/revised/NoDrug/*/')):
    date = f.split('/')[-2]
    for z in sorted(glob.glob(f+'*.zip')):
        img_header = z.split('/')[-1].split('_')[0]
        try:
            img_file = '{}{}.jpeg'.format(f,img_header)
            img = color.rgb2gray(plt.imread(img_file))
        except:
            img_file = '{}{}.jpg'.format(f,img_header)
            img = color.rgb2gray(plt.imread(img_file))
        filtered = mo.utils.bandpass.dual_bandpass(img,pixel_microns=1,
                                                   min_structure_scale=5,
                                                   max_structure_scale=1000)
        points = rr.read_roi_zip(z)
        sorted_coords = sort_gridpoints(points)
        projection(filtered,sorted_coords,
                   ref_points,output_name=dst+'Plate-0_Date-{}.tif'.format(date),
                   save=True,plot=False)

In [10]:
# process plate 1-4
for f in sorted(glob.glob('/Users/jz-rolling/Desktop/Size_calculation/Combo_Day1_Plating/*/')):
    date = f.split('/')[-2]
    for z in sorted(glob.glob(f+'*.zip')):
        img_header = z.split('/')[-1].split('_')[0]
        try:
            img_file = '{}{}.jpeg'.format(f,img_header)
            img = color.rgb2gray(plt.imread(img_file))
        except:
            img_file = '{}{}.jpg'.format(f,img_header)
            img = color.rgb2gray(plt.imread(img_file))
        filtered = mo.utils.bandpass.dual_bandpass(img,pixel_microns=1,
                                                   min_structure_scale=5,
                                                   max_structure_scale=1000)
        points = rr.read_roi_zip(z)
        sorted_coords = sort_gridpoints(points)
        projection(filtered,sorted_coords,ref_points,output_name=dst+'Plate-{}_Date-{}.tif'.format(int(img_header)+4,date),
                   save=True,plot=False)

In [11]:
# process plate 1-4
for f in sorted(glob.glob('/Users/jz-rolling/Desktop/Size_calculation/revised/Drug/*/')):
    date = f.split('/')[-2]
    for z in sorted(glob.glob(f+'*.zip')):
        img_header = z.split('/')[-1].split('_')[0]
        try:
            img_file = '{}{}.jpeg'.format(f,img_header)
            img = color.rgb2gray(plt.imread(img_file))
        except:
            img_file = '{}{}.jpg'.format(f,img_header)
            img = color.rgb2gray(plt.imread(img_file))
        filtered = mo.utils.bandpass.dual_bandpass(img,pixel_microns=1,
                                                   min_structure_scale=5,
                                                   max_structure_scale=1000)
        points = rr.read_roi_zip(z)
        sorted_coords = sort_gridpoints(points)
        projection(filtered,sorted_coords,ref_points,output_name=dst+'Plate-{}_Date-{}.tif'.format(img_header,date),
                   save=True,plot=False)

In [12]:
# convert to relative dates
start_dates = {0:1209,
               1:1202,
               2:1202,
               3:1202,
               4:1202,
               5:1203,
               6:1203,
               7:1203}
for f in sorted(glob.glob(dst+'*.tif')):
    plate,date = f.split('/')[-1].split('.tif')[0].split('_')
    starte_d = str(start_dates[int(plate[-1])])
    _d,month,day = date.split('-')
    dmonth = int(month)%int(starte_d[:2])
    dday = int(day)+31*dmonth-int(starte_d[2:])
    newname = '{}_{}_{}'.format(plate,date,dday)
    os.rename(f,dst+newname+'.tif')